In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Jerald\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Jerald\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from selenium.webdriver.common.keys import Keys
import praw
import pandas as pd
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [7]:
def scrape_url(source_path):
    page = 1
    url = []
    search_list = ['singapore','bivalent']
    while(page <= 2): 
        browser = webdriver.Chrome('chromedriver',options=chrome_options)
        browser.get(source_path)
        Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        source = browser.page_source
        soup = bs(source, 'html.parser')
        body = soup.find('body')

        threads = soup.find_all("header", class_="search-result-header")
        next_page = soup.find_all("span", class_="nextprev")
        source_path = next_page[0].find("a").get('href')
                    
        for link in threads:
            title = link.find("a").text
            if re.compile('(?=.*' + ')(?=.*'.join(search_list) + ')', re.IGNORECASE).search(title): #re.IGNORECASE is used to ignore case
                url_string = link.find('a').get('href')
                url.append(url_string)
        page += 1            
    return url

In [ ]:
source_path = "https://old.reddit.com/search?q=singapore+bivalent"
url_list = scrape_url(source_path)

In [3]:
reddit = praw.Reddit(client_id='', \
                     client_secret='', \
                     user_agent='', \
                     username='', \
                     password='')

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [4]:
topics_dict = { "author": [],
                "body": [],
                "score": [],
                "date": [],
                "source": []}

In [5]:
def get_date(created):
    original_date = dt.datetime.fromtimestamp(created)
    formatted_date = original_date.strftime("%b %Y")
    return formatted_date

In [6]:
def process_topic_data(submission):
    comments = []
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        if comment.body != "[removed]":
            timestamp = get_date(comment.created)
            topics_dict["author"].append(str(comment.author))
            topics_dict["body"].append(comment.body)
            topics_dict["score"].append(comment.score)
            topics_dict["date"].append(timestamp)
            topics_dict["source"].append("reddit")
    return comments

In [7]:
for link in url_list:
    submission = reddit.submission(url=link)
    process_topic_data(submission)

In [8]:
topics_data = pd.DataFrame(topics_dict)

In [9]:
topics_data = topics_data.dropna()

In [10]:
topics_data.to_json("reddit.json", orient='records')